# Lab 6:  BasicRNNCell

#### TensorFlow 0.9 implementation based on hunkim's tutorial
https://hunkim.github.io/ml/

https://www.youtube.com/watch?v=A8wJYfDUYCk&feature=youtu.be

https://gist.github.com/j-min/481749dcb853b4477c4f441bf7452195  에서 가져온 코드임. HunKim's tutorial으로 보고

j-min(https://gist.github.com/j-min) 님이 작성하신 코드임.

필요한 라이브러리를 import한다.

In [19]:
import tensorflow as tf
import numpy as np
import functions   # functions.showOperation()함수를 쓰기 위해 import함.

char_rdic은 특정 id에 대한 문자를 출력하는데 사용됨. 
예: 1이라는 id에 대한 문자를 출력하려면 print(char_rdic[1])를 실행하면 'e'라는 문자가 출력됨.

반면 char_dic은 특정 문자에 대한 id를 찾는데 사용됨. 
예: 'e'라는 문자에 대한 id를 출력하려면 print(char_dic['e'])를 실행하면 됨.


In [20]:
char_rdic = ['h', 'e', 'l', 'o'] # id -> char
char_dic = {w : i for i, w in enumerate(char_rdic)} # char -> id
print (char_dic)

{'o': 3, 'l': 2, 'h': 0, 'e': 1}


In [23]:
print(char_dic['e'])

1


'hello'라는 문자열에 있는 각 문자에 대한 id를 ground_truth배열에 저장함.

In [3]:
ground_truth = [char_dic[c] for c in 'hello']
print (ground_truth)

[0, 1, 2, 2, 3]


ground_truth[:-1]은 ground_truth배열의 처음부터 끝에서 두번째까지의 원소를 가진 배열임.

In [24]:
print(ground_truth[:-1])

[0, 1, 2, 2]


In [4]:
x_data = np.array([[1,0,0,0], # h
                   [0,1,0,0], # e
                   [0,0,1,0], # l
                   [0,0,1,0]], # l
                 dtype = 'f')

ground_truth[:-1]라는 배열에 대해 one-hot coding하여 x_data에 넣는다.

In [6]:
x_data = tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1)
print(x_data)

# functions.showOperation은 tf 연산을 세션에서 수행한 결과를 반환하는 함수임.

functions.showOperation(tf.one_hot(ground_truth[:-1], len(char_dic), 1.0, 0.0, -1))

Tensor("one_hot:0", shape=(4, 4), dtype=float32)
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]]


In [7]:
# rnn에 대한 설정
rnn_size = len(char_dic) # 4,입력이 아니라 rnn의 출력벡터의 크기임. 
                         # 즉, y벡터의 원소의 갯수임.
batch_size = 1      # 
output_size = 4

In [8]:
# RNN Model
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = rnn_size,
                                       input_size = None, # deprecated at tensorflow 0.9
                                       #activation = tanh,
                                       )
print(rnn_cell)

RNN의 셀 내부의 상태벡터들($h_0$, $h_1$,....)을 0으로 초기화한다:

In [9]:
initial_state = rnn_cell.zero_state(batch_size, tf.float32)
functions.showOperation(initial_state)

[[ 0.  0.  0.  0.]]


In [10]:
initial_state_1 = tf.zeros([batch_size, rnn_cell.state_size]) #  위 코드와 같은 결과
print(initial_state_1)

Tensor("zeros_1:0", shape=(1, 4), dtype=float32)


In [11]:
x_split = tf.split(0, len(char_dic), x_data) # 가로축으로 4개로 split
print(x_split)
functions.showOperation(tf.split(0, len(char_dic), x_data))
functions.showOperation(x_split[0])
"""
[[1,0,0,0]] # h
[[0,1,0,0]] # e
[[0,0,1,0]] # l
[[0,0,1,0]] # l
"""

[<tf.Tensor 'split:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:1' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:2' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:3' shape=(1, 4) dtype=float32>]
[array([[ 1.,  0.,  0.,  0.]], dtype=float32), array([[ 0.,  1.,  0.,  0.]], dtype=float32), array([[ 0.,  0.,  1.,  0.]], dtype=float32), array([[ 0.,  0.,  1.,  0.]], dtype=float32)]
[[ 1.  0.  0.  0.]]


'\n[[1,0,0,0]] # h\n[[0,1,0,0]] # e\n[[0,0,1,0]] # l\n[[0,0,1,0]] # l\n'

RNN모델을 정의함:

In [12]:
outputs, state = tf.nn.rnn(cell = rnn_cell, inputs = x_split, initial_state = initial_state)

In [13]:
functions.showOperation(outputs)
functions.showOperation(state)

[array([[ 0.10201149,  0.23352441,  0.23484631,  0.05947667]], dtype=float32), array([[ 0.60824579,  0.12183696,  0.01620867, -0.04638267]], dtype=float32), array([[-0.17860363, -0.30378503, -0.26180285, -0.08061511]], dtype=float32), array([[-0.47460279, -0.42061964, -0.03461161, -0.16280842]], dtype=float32)]
[[-0.05095752  0.49237123  0.26884806 -0.1694462 ]]


seq2seq에 들어갈 수 있는 형태로 output의 shape을 다시 정의함

In [14]:
logits = tf.reshape(tf.concat(1, outputs), # shape = 1 x 16
                    [-1, rnn_size])        # shape = 4 x 4
logits.get_shape()
functions.showOperation(logits)
"""
[[logit from 1st output],
[logit from 2nd output],
[logit from 3rd output],
[logit from 4th output]]
"""


[[-0.25952506 -0.49805069 -0.24455665 -0.09719572]
 [ 0.6316548  -0.33197194 -0.39149716 -0.65096629]
 [-0.22191067  0.05329145 -0.318876    0.40735328]
 [ 0.19110718 -0.17995055 -0.50942498 -0.194318  ]]


'\n[[logit from 1st output],\n[logit from 2nd output],\n[logit from 3rd output],\n[logit from 4th output]]\n'

참값을 가진 target도 shape를 재정의함. ( [1,2,2,3] 에서 [1 2 2 3]의 형태로)

In [15]:
targets = tf.reshape(ground_truth[1:], [-1]) # a shape of [-1] flattens into 1-D
targets.get_shape()
functions.showOperation(targets)

[1 2 2 3]


초기의 weight값들은 모두 1로 초기화. (입력과 곱해지는 weight값들)

In [16]:
weights = tf.ones([len(char_dic) * batch_size])
functions.showOperation(weights)

[ 1.  1.  1.  1.]


In [17]:
loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.RMSPropOptimizer(0.01, 0.9).minimize(cost)

학습을 시킨다.

In [18]:
# Launch the graph in a session
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        results = sess.run(logits)
        print(results)
        print(result, [char_rdic[t] for t in result])           

[[ 0.154439   -0.24573255 -0.07015415 -0.54687232]
 [ 0.02366722 -0.59855378 -0.06869259 -0.18825668]
 [-0.15818414  0.48190385 -0.59194183 -0.07254142]
 [-0.23743634  0.38949928 -0.32934746 -0.56906962]]
[0 0 1 1] ['h', 'h', 'e', 'e']
[[ 0.13791676 -0.23055612 -0.06934691 -0.55746192]
 [ 0.0024071  -0.59304094 -0.03673853 -0.21922804]
 [-0.17919061  0.48343214 -0.56471646 -0.08844101]
 [-0.25477496  0.39505142 -0.30809414 -0.56900471]]
[0 0 1 1] ['h', 'h', 'e', 'e']
[[ 0.12136459 -0.21534471 -0.06823723 -0.5677858 ]
 [-0.01856934 -0.58730561 -0.00399116 -0.24934888]
 [-0.19903572  0.4852367  -0.53551549 -0.10342944]
 [-0.27025655  0.40081587 -0.28618664 -0.5679388 ]]
[0 2 1 1] ['h', 'l', 'e', 'e']
[[ 0.10478518 -0.20009916 -0.06683713 -0.57783324]
 [-0.03927363 -0.5813666   0.02944974 -0.27855417]
 [-0.21777713  0.48727909 -0.50424796 -0.1174598 ]
 [-0.28391618  0.40684226 -0.26361367 -0.56580997]]
[0 2 1 1] ['h', 'l', 'e', 'e']
[[ 0.0881848  -0.18482505 -0.0651667  -0.58759403]
 [-0.